In this notebook we will perform classification on the Human activity recognition data
which you can find in this [link](https://www.kaggle.com/datasets/uciml/human-activity-recognition-with-smartphones). The database was built from the recordings of 30 study participants performing activities of daily living (ADL) while carrying a waist-mounted smartphone with embedded inertial sensors. The objective is to classify activities into one of the six activities performed. For this task you need to first download the data from aforementioned link and keep it in the `data` directory inside tutorials. Also, download the pandas library via `pip install pandas`

In [58]:
import pdb
import numpy as np
import pandas as pd 
import sys
sys.path.insert(0, '../')
from reservoirpy.nodes import Reservoir, Input, LinearRegression, RidgeRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from reservoirpy.datasets import mackey_glass
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [59]:
# First read the data using pandas and convert into numpy format
train = pd.read_csv("data/train.csv")
X_train= train.iloc[:,:-2].to_numpy()
Y_train= train.iloc[:,-1]
test= pd.read_csv("data/test.csv")
X_test=test.iloc[:,:-2].to_numpy()
Y_test= test.iloc[:,-1]

The labels i.e. Y_train and Y_test are in string format, thus, in order to convert them
into integers we will have to create a mapping between the string labels and their respective
class integers.

In [60]:
label2id = {"SITTING":0, "WALKING_DOWNSTAIRS":1,
"WALKING_UPSTAIRS":2, "STANDING":3, "WALKING":4,
"LAYING":5
}
Y_train = np.array([label2id[Y_train[i]] for i in range(len(Y_train))])
Y_test = np.array([label2id[Y_test[i]] for i in range(len(Y_test))])

Now, the target labels are in integer format, howvwer since we are performing regression, 
the output labels need to be in vectors. Thus we perform one-hot encoding on the training labels

In [61]:
print(f" Classes: {set(Y_train)} ")
Y_train_onehot = np.zeros((Y_train.shape[0], len(set(Y_train))))
for i in range(len(Y_train)):
	Y_train_onehot[i, Y_train[i]] = 1
Y_train = Y_train_onehot

 Classes: {0, 1, 2, 3, 4, 5} 


Now, we will create the model

In [49]:
source = Input()
reservoir = Reservoir(500, sr=0.9, lr=0.1)
readout = RidgeRegression()
model = source >> reservoir >> readout

We train the model by taking the last state from the reservoir node and storing it into lists.
Finally we call our fit method on the RidgeRegression readout node on the last satetes and target labels.

In [16]:
states_train = []
for x in X_train:
    states = reservoir.run(x, reset=True)
    states_train.append(states[-1, np.newaxis])
readout.fit(np.array(states_train).squeeze(), np.array(Y_train).squeeze())

Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 2833.99it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 2288.22it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 3088.59it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 2882.68it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 2267.19it/s]


...

Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 3194.44it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 3302.60it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 2931.03it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 1586.95it/s]


Running Reservoir-1:   0%|          | 0/1 [00:00<?, ?it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Finally, we perform prediction on the test data and measure the accuracy.

In [17]:
states_test = []
for x in X_test:
    states = reservoir.run(x, reset=True)
    states_test.append(states[-1, np.newaxis])

Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 2857.16it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 2487.72it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 2597.09it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 3223.91it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 3368.92it/s]


...

Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 2438.55it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 606.81it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 3816.47it/s]


Running Reservoir-1: 100%|██████████| 1/1 [00:00<00:00, 3236.35it/s]


Running Reservoir-1:   0%|          | 0/1 [00:00<?, ?it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [19]:
y_pred = readout.run(np.array(states_test).squeeze())
Y_pred_class = np.argmax(y_pred, axis=1)


Running RidgeRegression-1: 100%|██████████| 2947/2947 [00:00<00:00, 19333.46it/s]


In [25]:
score = accuracy_score(Y_test, Y_pred_class)
print("Accuracy: ", f"{score * 100:.3f} %")

Accuracy:  91.517 %


### Comparison with Native Ridge Node

In [62]:
from reservoirpy.nodes import Ridge
source = Input()
reservoir = Reservoir(500, sr=0.9, lr=0.1)
readout = Ridge(ridge=1e-3)
model = source >> reservoir >> readout

In [63]:
X_train = [X_train[i] for i in range(X_train.shape[0])]
X_test = [X_test[i] for i in range(X_test.shape[0])]

In [64]:
X_train[0].shape

(561,)

In [65]:
states_train = []
for x in X_train:
    states = reservoir.run(x, reset=True)
    states_train.append(states[-1, np.newaxis])
readout.fit(np.array(states_train).squeeze(), np.array(Y_train).squeeze())

Running Reservoir-5: 100%|██████████| 1/1 [00:00<00:00, 1763.05it/s]


Running Reservoir-5: 100%|██████████| 1/1 [00:00<00:00, 3398.95it/s]


Running Reservoir-5: 100%|██████████| 1/1 [00:00<00:00, 2898.62it/s]


Running Reservoir-5: 100%|██████████| 1/1 [00:00<00:00, 2668.13it/s]


Running Reservoir-5: 100%|██████████| 1/1 [00:00<00:00, 956.95it/s]


...

Running Reservoir-5: 100%|██████████| 1/1 [00:00<00:00, 1968.23it/s]


Running Reservoir-5: 100%|██████████| 1/1 [00:00<00:00, 3045.97it/s]


Running Reservoir-5: 100%|██████████| 1/1 [00:00<00:00, 2092.97it/s]


Running Reservoir-5: 100%|██████████| 1/1 [00:00<00:00, 2537.39it/s]


Running Reservoir-5: 100%|██████████| 1/1 [00:00<00:00, 2053.01it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [54]:
X_test = [X_test[i] for i in range(X_test.shape[0])]
states_test = []
for x in X_test:
    states = reservoir.run(x, reset=True)
    states_test.append(states[-1, np.newaxis])

Running Reservoir-4: 100%|██████████| 1/1 [00:00<00:00, 1424.21it/s]


Running Reservoir-4: 100%|██████████| 1/1 [00:00<00:00, 2288.22it/s]


Running Reservoir-4: 100%|██████████| 1/1 [00:00<00:00, 1893.59it/s]


Running Reservoir-4: 100%|██████████| 1/1 [00:00<00:00, 1013.12it/s]


Running Reservoir-4: 100%|██████████| 1/1 [00:00<00:00, 3289.65it/s]


...

Running Reservoir-4: 100%|██████████| 1/1 [00:00<00:00, 2283.24it/s]


Running Reservoir-4: 100%|██████████| 1/1 [00:00<00:00, 2513.06it/s]


Running Reservoir-4: 100%|██████████| 1/1 [00:00<00:00, 2799.94it/s]


Running Reservoir-4: 100%|██████████| 1/1 [00:00<00:00, 3806.08it/s]


Running Reservoir-4: 100%|██████████| 1/1 [00:00<00:00, 3401.71it/s]


In [55]:
y_pred = readout.run(np.array(states_test).squeeze())
Y_pred_class = np.argmax(y_pred, axis=1)

Running Ridge-1: 100%|██████████| 2947/2947 [00:00<00:00, 40833.59it/s]


In [57]:
score = accuracy_score(Y_test, Y_pred_class)
print("Accuracy: ", f"{score * 100:.3f} %")

Accuracy:  91.788 %
